In [ ]:
import os
os.chdir("..")

In [ ]:
import requests
import pprint
import pandas as pd
from src.common.db_schema import SQLItem, Base
from src.common.schema import SimSearchClassificationRequest, GetStatusRequest
from src.services.sql_service import SQLService
from src.endpoints import *

# Prepare DB

In [ ]:
Base.metadata.create_all(SQLService.get_engine())
session = SQLService.get_session()

# Load items to classify

In [ ]:
test_df = pd.read_csv("resources/test_df.csv")
test_df.sample(10)

In [ ]:
items = [
    SQLItem(
        season=test_df.iloc[i]["season"],
        origin_id=test_df.iloc[i]["item_id"],
        supplier_name=test_df.iloc[i]["supplier_name"],
        supplier_reference_description=test_df.iloc[i]["supplier_reference_description"],
        materials=test_df.iloc[i]["materials"]
    )
    for i in range(len(test_df))
]
SQLService.insert(items)

# Trigger python endpoint

In [ ]:
response_class = requests.post(
    url="http://127.0.0.1:8000/sim_search_classification",
    json=SimSearchClassificationRequest(item_ids=[62677]).model_dump()
)
print(response_class.status_code)
pprint.pp(response_class.json())

# Check task status

In [ ]:
response = requests.post(
    url="http://127.0.0.1:8000/get_status",
    json=GetStatusRequest(task_id=response_class.json()["task_id"]).model_dump()
)
print(response.status_code)
pprint.pp(response.json())